# 使用6个波段的Landset数据来训练简化后的VGG模型，使用了fit_generator()
# 挺好，这个批量训练就是每次从硬盘上拿出一批数据放入模型训练，
# 影像和标签数据合在一起为tuple()，这样批量训练就很方便了。

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import random
from tqdm import tqdm
import random
import PIL.Image as img
# from skimage.io import imread
from skimage.transform import resize
from skimage.io import imread
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing import image
from keras.utils import multi_gpu_model
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)
import matplotlib.pyplot as plt
plt.style.use("ggplot")


# path = "H:/工作站迁移环境/多波段数据集/total_data/" 
path = "G:/DL-tools/工作站迁移环境/多波段数据集/total_data/"
path_1 = path + "landset-1-3/"
path_2 = path + "landset-4-6/"
# path_3 = path + "dem-1-3/"
# path_4 = path + "dem-4-6/"

bounds = 6  # 加载几个波段的数据
num_aug = 2  # 使用几种数据增强，默认减一
classes = 2
batch_size = 8


Using TensorFlow backend.


### 使用 label_generator = []，加载的标签数据集是影像数据的2倍，为什么？

In [2]:
def trainGenerator(batch_size=None, train_image_path=None, shape=None, bounds=None):
    imageList = os.listdir(train_image_path + "landset-1-3/")
    img_generator = np.zeros((batch_size, shape, shape, bounds), np.float32)
    label_generator = np.zeros((batch_size, 2))
    # label_generator = []
    # for i in range(1):
    while(True):
        rand = random.randint(0, len(imageList) - batch_size)
        for j in range(batch_size):
            img1 = imread(train_image_path + "landset-1-3/" + imageList[rand+j])
            img2 = imread(train_image_path + "landset-4-6/" + imageList[rand+j])
#             img3 = imread(train_image_path + "dem-1-3/" + imageList[rand+j])
#             img4 = imread(train_image_path + "dem-4-6/" + imageList[rand+j])
            img_1 = resize(img1, (224, 224, 3), mode='constant', preserve_range=True)
            img_2 = resize(img2, (224, 224, 3), mode='constant', preserve_range=True)
#             img_3 = resize(img3, (224, 224, 3), mode='constant', preserve_range=True)
#             img_4 = resize(img4, (224, 224, 3), mode='constant', preserve_range=True)
            img_generator[j,:,:,:3] = img_1 # / 255
            img_generator[j,:,:,3:6] = img_2 # / 255 
#             img_generator[j,:,:,6:9] = img_3 # / 255
#             img_generator[j,:,:,9:] = img_4 # / 255
            temp = os.path.splitext(imageList[rand+j])[0]
            if int(temp) < 2500:
                # label_generator.append([0,1])
                # label_generator[j,1] = 1
                label_generator[j] = (0,1)
            else:
                # label_generator.append([1,0])
                # label_generator[j,0] = 1
                label_generator[j] = (1,0)
        # img_generator = process(img_generator)
        yield (img_generator, np.array(label_generator))


In [3]:
# for i in  trainGenerator(batch_size=batch_size, train_image_path=path, shape=224, bounds=bounds):
#     print(i[1])

In [4]:
def get_model(cl=2):
    model = Sequential()
    classes = cl

    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, bounds)))
    model.add(BatchNormalization())
    # model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), activation='relu'))
    # model.add(Conv2D(256, (3, 3), activation='relu'))
    # model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(512, (3, 3), activation='relu'))
    # model.add(Conv2D(512, (3, 3), activation='relu'))
    # model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(512, (3, 3), activation='relu'))
    # model.add(Conv2D(512, (3, 3), activation='relu'))
    # model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    # model.add(Dense(4096, activation="relu"))
    model.add(Dense(1024, activation="relu"))

    model.add(Dense(classes, activation='softmax'))
    return model


# 控制模型训练过程
callback = [
    EarlyStopping(patience=5, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    ModelCheckpoint('G:/6_landset_aug.h5', verbose=1, monitor='val_loss', save_best_only=True, save_weights_only=True),
    # TensorBoard(log_dir='9bound_log', histogram_freq=1)
    # plot_losses
]

In [5]:
aug = True
if aug:
    epoch=100
    train_set = trainGenerator(batch_size=batch_size, train_image_path=path, shape=224, bounds=bounds)
    val_set = trainGenerator(batch_size=batch_size, train_image_path=path, shape=224, bounds=bounds)
    train_num = len(os.listdir(path_1))
    #  训练集每个epoch有多少个batch_size
    steps_per_epoch = int(train_num / batch_size)
    #  验证集每个epoch有多少个batch_size
    # validation_steps = train_num / batch_size
    model1 = get_model(cl=classes)
    sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
    adm = Adam()
    model1.compile(loss='binary_crossentropy', optimizer=sgd, metrics=["accuracy"])
    model1.summary()
#     history = model1.fit_generator(train_set,
#                         steps_per_epoch = steps_per_epoch,
#                         epochs = epoch,
#                         callbacks = callback,
#                         validation_data = val_set,
#                         validation_steps =  steps_per_epoch)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 64)      3520      
_________________________________________________________________
batch_normalization_1 (Batch (None, 222, 222, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 107, 107, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 51, 51, 256)       295168    
__________

In [6]:
history = model1.fit_generator(train_set,
                    steps_per_epoch = steps_per_epoch,
                    epochs = epoch,
                    callbacks = callback,
                    validation_data = val_set,
                    validation_steps = 5)

Epoch 1/100
382/382 [==============================] - 168s 439ms/step - loss: 0.6604 - acc: 0.6587 - val_loss: 0.4094 - val_acc: 1.0000

Epoch 00001: val_loss improved from inf to 0.40938, saving model to G:/6_landset_aug.h5
Epoch 2/100
382/382 [==============================] - 165s 432ms/step - loss: 0.6240 - acc: 0.7055 - val_loss: 1.0016 - val_acc: 0.4500

Epoch 00002: val_loss did not improve
Epoch 3/100
382/382 [==============================] - 165s 432ms/step - loss: 0.6380 - acc: 0.6839 - val_loss: 0.7923 - val_acc: 0.6500

Epoch 00003: val_loss did not improve
Epoch 4/100
230/382 [=================>............] - ETA: 1:04 - loss: 0.6268 - acc: 0.6984

KeyboardInterrupt: 